# Elo for csgo teams

In [1]:
import pandas as pd
import math

In [6]:
match_online = pd.read_csv('data/matches_Online_21.csv',sep=';')
match_lan = pd.read_csv('data/matches_Lan_21.csv',sep=';')
match_big_events = pd.read_csv('data/matches_BigEvents_21.csv',sep=';')
match_majors = pd.read_csv('data/matches__Majors_21.csv',sep=';')
match_history  = pd.concat([match_majors,match_big_events,match_lan,match_online])
match_history.drop("Unnamed: 0",axis=1,inplace=True)

In [28]:
match_history.head()

,date,team1,team2,map,event,event_type,event_tier,s1,s2
0,2012-11-24,Ninjas in Pyjamas,VeryGames,Inferno_se,DreamHack Winter 2012,BigEvents,S,16,1
1,2012-11-24,VeryGames,Ninjas in Pyjamas,Dust2_se,DreamHack Winter 2012,BigEvents,S,9,16
2,2012-11-24,mousesports,Ninjas in Pyjamas,Nuke_se,DreamHack Winter 2012,BigEvents,S,5,16
3,2012-11-24,Ninjas in Pyjamas,mousesports,Dust2_se,DreamHack Winter 2012,BigEvents,S,16,14
4,2012-11-24,VeryGames,Lemondogs,Inferno_se,DreamHack Winter 2012,BigEvents,S,16,9


## Clean data

In [8]:
#renaming collumns
match_history.rename(columns={'Date':'date', 'Team1':"team1", 'Team2':"team2", 'Map':"map", 'Event':"event"},inplace=True)
#separating score and name to get a clean data
match_history[['team1','s1']] = match_history['team1'].str.split('(',1,expand=True)
match_history[['team2','s2']] = match_history['team2'].str.split('(',1,expand=True)
match_history['team1'] = match_history['team1'].str.strip()
match_history['team2'] = match_history['team2'].str.strip()
match_history['s1'] = match_history['s1'].str[:-1]
match_history['s2'] = match_history['s2'].str[:-1]
#separating the abreviation of the map
match_history['map'] = match_history['map'].str[4:]
# Score were not properly score
match_history.loc[match_history['s2'] == "-",'s2'] = 16

### Removing duplicates

In [29]:
match_history.drop_duplicates(subset=match_history.columns.difference(['event_type']),keep='last',inplace=True)

## Change type

### Score

In [27]:
match_history['s1'] = pd.to_numeric(match_history['s1'])
match_history['s2'] = pd.to_numeric(match_history['s2'])

### date

In [30]:
match_history['date'] = pd.to_datetime(match_history['date'])


## Adding information for the table

## Winner

In [31]:
match_history['winner'] = (match_history['s1'] - match_history['s2']>0)

## Elo collumn

In [32]:
match_history['elo1'] = 1500    
match_history['elo2'] = 1500

## Index

In [33]:
match_history.sort_values(by=["date"],inplace=True)
match_history.reset_index(drop=True,inplace=True)
match_history['index1'] = match_history.index

In [34]:
match_history.head(20)

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
0,2012-01-11,CPLAY,dotpiXels,Mirage_ce,ESL Pro Series Germany Winter Season 2012,Online,A,6,16,False,1500,1500,0
1,2012-01-11,gamed!de,iPLAY,Train_se,ESL Pro Series Germany Winter Season 2012,Online,A,16,7,True,1500,1500,1
2,2012-01-11,CPLAY,ALTERNATE aTTaX,Train_se,ESL Pro Series Germany Winter Season 2012,Online,A,9,16,False,1500,1500,2
3,2012-01-12,myDGB.net,VeryGames,Nuke_se,AMD Sapphire CS:GO Invitational,Lan,A,16,12,True,1500,1500,3
4,2012-01-12,ESC,BuyKey,Inferno_se,AMD Sapphire CS:GO Invitational,Lan,A,7,16,False,1500,1500,4
5,2012-01-12,Ninjas in Pyjamas,ESC,Train_se,AMD Sapphire CS:GO Invitational,Lan,A,16,12,True,1500,1500,5
6,2012-01-12,eXtensive,k1ck,Inferno_se,AMD Sapphire CS:GO Invitational,Lan,A,14,16,False,1500,1500,6
7,2012-01-12,VeryGames,dAT,Nuke_se,AMD Sapphire CS:GO Invitational,Lan,A,16,13,True,1500,1500,7
8,2012-01-12,myDGB.net,eXtensive,Nuke_se,AMD Sapphire CS:GO Invitational,Lan,A,15,15,False,1500,1500,8
9,2012-01-12,eXtensive,VeryGames,Inferno_se,AMD Sapphire CS:GO Invitational,Lan,A,14,16,False,1500,1500,9


# Applying elo function

In [43]:
def eloretriver(team,index):
    #retrive the last elo point from a data frame.
    last_elo = match_history[(match_history['index1'] < index) & ((match_history['team1'] == team)|(match_history['team2'] == team))].tail(1)
    if len(last_elo) == 0:
        return None
    elif last_elo['team1'].iloc[0] == team:
        print(last_elo['index1'].iloc[0])
        return last_elo['elo1'].iloc[0]
    elif last_elo['team2'].iloc[0] == team:
        print(last_elo['index1'].iloc[0])
        return last_elo['elo2'].iloc[0]

def Probwin(rating1,rating2):
    # Function to calculate the Probability
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

def EloRatingcalculator(team1,team2,index1,winner,k):
    Ra = eloretriver(team1,index1)
    Rb = eloretriver(team2,index1)
    # If team has no rating the starting rating is 1500(?change)
    if Ra is None:
        Ra = 1500
    if Rb is None:
        Rb = 1500
    # setting k value
    if k == 'Online':
        k = 5
    if k == 'Lan':
        k = 10
    if k == 'BigEvents':
        k = 20
    if k == 'Majors':
        k = 30
    print(k)
    # Function to calculate Elo rating
    # K is a constant.
    # winner determines whether Player A wins or Player B
    # To calculate the Winning
    # Probability of Player B
    print(Ra)
    print(Rb)
    Pb = Probwin(Ra, Rb)
    # To calculate the Winning
    # Probability of Player A
    Pa = Probwin(Rb, Ra)
      # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (winner == True) :
        elo1 = Ra + k * (1 - Pa)
        elo2 = Rb + k * (0 - Pb)
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    else :
        elo1 = Ra + k * (0 - Pa)
        elo2 = Rb + k * (1 - Pb)
    print(elo1)
    print(elo2)
    return pd.Series([round(elo1),round(elo2)])
    

K-factor  	Used for players with ratings ...
k=40 for a player new to the rating list until the completion of events with a total of 30 games, and for all players until their 18th birthday, as long as their rating remains under 2300.
k=20 	for players who have always been rated under 2400.
k=10for players with any published rating of at least 2400 and at least 30 games played in previous events. Thereafter it remains permanently at 10.

Score after mathces,series or lan torunaments(greater difference for tournaments?? Performance Rating by tournament?)

elo starting at what?

elo for players?

In [45]:
match_history[['elo1','elo2']] = match_history.apply(lambda x : EloRatingcalculator(x['team1'],x['team2'],x['index1'],x['winner'],x['event_type']),axis=1)

5
1500
1500
1497.5
1502.5
5
1500
1500
1502.5
1497.5
0
5
1485
1500
1482.6078666652556
1502.3921333347444
10
1500
1500
1505.0
1495.0
10
1500
1500
1495.0
1505.0
4
10
1500
1485
1504.7842666694885
1480.2157333305115
10
1500
1500
1495.0
1505.0
3
10
1485
1500
1490.2157333305115
1494.7842666694885
3
6
10
1515
1485
1509.5693350797778
1490.4306649202222
8
7
10
1515
1515
1510.0
1520.0
4
10
1515
1500
1519.7842666694885
1495.2157333305115
6
8
10
1515
1485
1519.5693350797778
1480.4306649202222
7
9
10
1485
1485
1480.0
1490.0
12
11
10
1485
1485
1480.0
1490.0
9
11
10
1515
1515
1520.0
1510.0
13
14
10
1485
1485
1480.0
1490.0
10
10
1500
1485
1504.7842666694885
1480.2157333305115
5
16
10
1515
1515
1520.0
1510.0
16
5
10
1485
1485
1480.0
1490.0
17
10
10
1515
1515
1520.0
1510.0
17
19
10
1485
1485
1490.0
1480.0
19
18
10
1515
1485
1519.5693350797778
1480.4306649202222
10
1500
1500
1505.0
1495.0
20
18
10
1515
1515
1510.0
1520.0
22
22
10
1485
1515
1480.4306649202222
1519.5693350797778
5
1500
1500
1497.5
1502.5
25

KeyboardInterrupt: 

In [39]:
match_history.query('team1 == "MIBR" | team2 == "MIBR"').head(50)

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
43288,2018-01-09,Astralis,MIBR,Train,DreamHack Masters Stockholm 2018,Lan,A,16,14,True,1515,1485,43288
43289,2018-01-09,Astralis,MIBR,Overpass,DreamHack Masters Stockholm 2018,Lan,A,16,6,True,1515,1485,43289
43298,2018-01-09,Astralis,MIBR,Overpass,DreamHack Masters Stockholm 2018,BigEvents,S,16,6,True,1515,1485,43298
43299,2018-01-09,Astralis,MIBR,Train,DreamHack Masters Stockholm 2018,BigEvents,S,16,14,True,1515,1485,43299
44607,2018-02-10,MIBR,Bravado,Inferno,ECS Season 6 North America Challenger Cup,Online,A,16,5,True,1515,1485,44607
44608,2018-02-10,MIBR,Bravado,Cache,ECS Season 6 North America Challenger Cup,Online,A,16,8,True,1515,1485,44608
44613,2018-02-10,MIBR,TeamOne,Inferno,ECS Season 6 North America Challenger Cup,Online,A,16,5,True,1515,1485,44613
44638,2018-02-10,TeamOne,MIBR,Train,ECS Season 6 North America Challenger Cup,Online,A,12,16,False,1485,1515,44638
44650,2018-02-11,Cloud9,MIBR,Inferno,BLAST Pro Series Copenhagen 2018,Lan,A,16,13,True,1515,1485,44650
44698,2018-02-11,Astralis,MIBR,Inferno,BLAST Pro Series Copenhagen 2018,BigEvents,S,16,14,True,1515,1485,44698


In [40]:
EloRatingcalculator('Renegades','MIBR',46008,False,30)

45905
44714
1485
1485


0    1470
1    1500
dtype: int64

In [37]:
match_history.to_csv('data/maths_elo.csv',sep = ';')